In [6]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset6/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset6/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset6/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset6/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(66542, 128, 12) (66542, 2) (7600, 128, 12) (7600, 2)


In [7]:
from numpy import mean
from numpy import std
from numpy import dstack
from numpy import save, load
from pandas import read_csv
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,TimeDistributed, MaxPooling1D
from matplotlib import pyplot
import tensorflow as tf

In [8]:
# reshape data into time steps of sub-sequences
n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
n_steps, n_length = 4, 32
x_train = x_train.reshape((x_train.shape[0], n_steps, n_length, n_features))
x_test = x_test.reshape((x_test.shape[0], n_steps, n_length, n_features))

In [11]:
from sklearn.model_selection import train_test_split
x_training, x_validation, y_training, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [12]:
model2 = Sequential()
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model2.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model2.add(TimeDistributed(Dropout(0.5)))
model2.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model2.add(TimeDistributed(Flatten()))
model2.add(LSTM(100))
model2.add(Dropout(0.5))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(n_outputs, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()
callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights= True)]


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, None, 30, 64)     2368      
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 28, 64)     12352     
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, None, 28, 64)     0         
 tributed)                                                       
                                                                 
 time_distributed_3 (TimeDis  (None, None, 14, 64)     0         
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, None, 896)        0

In [13]:
# fit network
history = model2.fit(x_training, y_training, epochs=200, verbose=True, validation_data=(x_validation, y_validation), batch_size=100)

Epoch 1/200
533/533 [==============================] - 19s 11ms/step - loss: 0.3118 - accuracy: 0.8632 - val_loss: 0.2602 - val_accuracy: 0.8823
Epoch 2/200
533/533 [==============================] - 5s 9ms/step - loss: 0.2203 - accuracy: 0.9094 - val_loss: 0.1989 - val_accuracy: 0.9147
Epoch 3/200
533/533 [==============================] - 5s 9ms/step - loss: 0.1849 - accuracy: 0.9249 - val_loss: 0.1677 - val_accuracy: 0.9305
Epoch 4/200
533/533 [==============================] - 4s 8ms/step - loss: 0.1612 - accuracy: 0.9360 - val_loss: 0.1571 - val_accuracy: 0.9349
Epoch 5/200
533/533 [==============================] - 4s 8ms/step - loss: 0.1400 - accuracy: 0.9449 - val_loss: 0.1411 - val_accuracy: 0.9432
Epoch 6/200
533/533 [==============================] - 4s 8ms/step - loss: 0.1226 - accuracy: 0.9516 - val_loss: 0.1411 - val_accuracy: 0.9445
Epoch 7/200
533/533 [==============================] - 5s 9ms/step - loss: 0.1138 - accuracy: 0.9563 - val_loss: 0.1375 - val_accuracy: 0.94

In [14]:
# evaluate model
loss, accuracy = model2.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy))
loss, accuracy = model2.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy))


Validation Accuracy:  0.9632
Testing Accuracy: 0.9076
